In [1]:
from modules.KerasCnnWithSpectralPool import CNN_Spectral_Pool
from modules.Utils import load_cifar10, load_cifar100
import matplotlib.pylab as plt
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)
#tf.autograph.set_verbosity(3)
import numpy as np
import IPython
import time
%matplotlib inline
%load_ext autoreload
%autoreload 2
# print(IPython.sys_info())
print(tf.__version__)

2.4.0


# Download CIFAR-100 data, if necessary, and load it

In [2]:
xtrain, ytrain, xtest, ytest = load_cifar10(5, get_test_data=True, channels_last=False)
xtrain = np.moveaxis(xtrain, 1, -1)
print(xtrain.shape)
xtest = np.moveaxis(xtest, 1, -1)
print(xtest.shape)

file already downloaded..
getting batch 1
getting batch 2
getting batch 3
getting batch 4
getting batch 5
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [3]:
xtrain, ytrain, xtest, ytest = load_cifar100(get_test_data=True, channels_last=False)
xtrain = np.moveaxis(xtrain, 1, -1)
print(xtrain.shape)
xtest = np.moveaxis(xtest, 1, -1)
print(xtest.shape)

file already downloaded..
(50000, 32, 32, 3)
(10000, 32, 32, 3)


# Subsample the data for hyperparameter search

In [4]:
sample_size = 2 ** 14
validation_size = 2 ** 10
# Randomly sample the x train data and y train data-------------------------------------------------------------------
random_sample = np.random.choice(np.arange(xtrain.shape[0]),
                             size=sample_size, replace=False)
xtrain_random_sampled = xtrain[random_sample]
ytrain_random_sampled = ytrain[random_sample]
# Separate them to train sample and validation sample-----------------------------------------------------------------
# For train
xtrain_sample = xtrain_random_sampled[:-validation_size]
ytrain_sample = ytrain_random_sampled[:-validation_size]
# For validation
xvalid_sample = xtrain_random_sampled[-validation_size:]
yvalid_sample = ytrain_random_sampled[-validation_size:]
# Display the prepared data
print(xtrain_sample.shape, ytrain_sample.shape)
print(xvalid_sample.shape, yvalid_sample.shape)
print(xtrain_sample[0].shape)  # Each x train sample is a rgb picture with size 32*32*3
print(ytrain_sample[0])  # Each y train sample is an integar indicating the category
print(type(ytrain_sample[0]))
num_classes = 10
print(tf.one_hot(indices=ytrain_sample, depth=num_classes))

(15360, 32, 32, 3) (15360,)
(1024, 32, 32, 3) (1024,)
(32, 32, 3)
80
<class 'numpy.int64'>
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(15360, 10), dtype=float32)


# Define ranges for hyperparameters

In [5]:
learning_rate_range = [1e-6,1e-4]
l2_norm_range = [5e-3, 1e-2]
M_range = [3, 6]
epsilon_range = [1e-7 , 1e-5]

# Perform the hyperparameter search

In [6]:
# Initialize variables to store parameters
best_train_accuracies = []
best_valid_accuracies = []
epochs = 100
hyperparams = []
search_times = 20
num_classes = 100

# For each search round
for search_idx in range(search_times):
    start_time=time.time()
    print('\nNo.{0} hyperparameters search started'.format(search_idx+1))
    # Randomly select hyperparameters learning rate, L2_norm, M, epsilon in the range respectively
    learning_rate = np.exp(np.random.uniform(low=np.log(learning_rate_range[0]), high=np.log(learning_rate_range[1])))
    l2_norm = np.exp(np.random.uniform(low=np.log(l2_norm_range[0]), high=np.log(l2_norm_range[1])))
    M = np.random.randint(low = M_range[0], high = M_range[1]) # Total number of convolution&spectral-pool layer-pairs
    epsilon = np.random.uniform(low = epsilon_range[0], high = epsilon_range[1])
    # Record selected hyperparameters l,L2,M,gamma respectively
    hyperparams.append({
        'learning_rate': learning_rate,
        'l2_norm': l2_norm,
        'M': M,
        'epsilon': epsilon
    })
    print ('Trying Hyperparameters:', hyperparams[-1])

    # Build the model-------------------------------------------------------------------------------------------------
    model = CNN_Spectral_Pool(M=M,l2_norm=l2_norm, num_classes=100)
    for layer in model.layers:
        layer.trainable = True
    # Instantiate hyperparameters-------------------------------------------------------------------------------------
    # Instantiate an optimizer
    adam = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=epsilon)
    # Instantiate a loss function.
    loss_fn = tf.keras.losses.CategoricalCrossentropy()
    # Instantiate a training accuracy metric
    train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
    # Instantiate a validation accuracy metric
    val_acc_metric = tf.keras.metrics.CategoricalAccuracy()
    # Compile the model-----------------------------------------------------------------------------------------------
    # model.compile(optimizer=sgd, loss=loss_fn, metrics=train_acc_metric)
    model.compile(optimizer=adam, loss=loss_fn, metrics=['accuracy'])
    # Fit the model---------------------------------------------------------------------------------------------------
    history = model.fit(xtrain_sample, tf.one_hot(indices=ytrain_sample, depth=num_classes), validation_data=(xvalid_sample, tf.one_hot(indices=yvalid_sample, depth=num_classes)), batch_size=256, epochs=epochs, verbose=1)
    
    # Record best performances----------------------------------------------------------------------------------------
    #train_loss=history.history['loss']
    #val_loss=history.history['val_loss']
    train_acc=history.history['accuracy']
    valid_acc=history.history['val_accuracy']
    best_train_accuracies.append(np.max(train_acc))
    best_valid_accuracies.append(np.max(valid_acc))
    # Calculate time used per search round
    end_time=time.time()
    time_taken = "%.2f" % (end_time-start_time)
    # Print one round search result
    print('No.{0} hyperparameters search finished!!!, it takes {1} seconds'.format(search_idx+1, time_taken))
    print('Current Best Train Accuracy: ', np.max(best_train_accuracies))
    print('Current Best Valid Accuracy: ', np.max(best_valid_accuracies))
    print('Current Best Hyperparameters: ', hyperparams[best_valid_accuracies.index(np.max(best_valid_accuracies))])

# Print the overall result during whole parameters search
Best_Train_Accuracy = np.max(best_train_accuracies)
Best_Valid_Accuracy = np.max(best_valid_accuracies)
Best_Hyperparameters = hyperparams[best_valid_accuracies.index(np.max(best_valid_accuracies))]
print('\nHyperparameters Search Ended!!!')
print('Best Train Accuracy Ever: ', Best_Train_Accuracy)
print('Best Valid Accuracy Ever: ', Best_Valid_Accuracy)
print('Best Hyperparameters Ever: ', Best_Hyperparameters)


No.1 hyperparameters search started
Trying Hyperparameters: {'learning_rate': 1.1721977220961097e-05, 'l2_norm': 0.007887309617732078, 'M': 4, 'epsilon': 9.647704264980253e-06}
Epoch 1/100
60/60 [==============================] - 3s 14ms/step - loss: 5.8184 - accuracy: 0.0117 - val_loss: 4.7552 - val_accuracy: 0.0127
Epoch 2/100
60/60 [==============================] - 0s 5ms/step - loss: 4.7106 - accuracy: 0.0197 - val_loss: 4.6355 - val_accuracy: 0.0195
Epoch 3/100
60/60 [==============================] - 0s 5ms/step - loss: 4.6296 - accuracy: 0.0291 - val_loss: 4.6153 - val_accuracy: 0.0332
Epoch 4/100
60/60 [==============================] - 0s 4ms/step - loss: 4.6120 - accuracy: 0.0387 - val_loss: 4.6068 - val_accuracy: 0.0410
Epoch 5/100
60/60 [==============================] - 0s 4ms/step - loss: 4.6020 - accuracy: 0.0501 - val_loss: 4.6016 - val_accuracy: 0.0479
Epoch 6/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5944 - accuracy: 0.0532 - val_loss: 4.596

60/60 [==============================] - 0s 4ms/step - loss: 4.5318 - accuracy: 0.0782 - val_loss: 4.5482 - val_accuracy: 0.0615
Epoch 12/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5221 - accuracy: 0.0820 - val_loss: 4.5423 - val_accuracy: 0.0635
Epoch 13/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5125 - accuracy: 0.0838 - val_loss: 4.5352 - val_accuracy: 0.0654
Epoch 14/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5030 - accuracy: 0.0878 - val_loss: 4.5286 - val_accuracy: 0.0693
Epoch 15/100
60/60 [==============================] - 0s 4ms/step - loss: 4.4926 - accuracy: 0.0921 - val_loss: 4.5226 - val_accuracy: 0.0674
Epoch 16/100
60/60 [==============================] - 0s 4ms/step - loss: 4.4893 - accuracy: 0.0916 - val_loss: 4.5166 - val_accuracy: 0.0762
Epoch 17/100
60/60 [==============================] - 0s 4ms/step - loss: 4.4804 - accuracy: 0.0954 - val_loss: 4.5117 - val_accuracy: 0.0762
Epoch 18/100
60/60 

Epoch 69/100
60/60 [==============================] - 0s 4ms/step - loss: 4.1912 - accuracy: 0.2027 - val_loss: 4.3253 - val_accuracy: 0.1465
Epoch 70/100
60/60 [==============================] - 0s 4ms/step - loss: 4.1770 - accuracy: 0.2086 - val_loss: 4.3233 - val_accuracy: 0.1406
Epoch 71/100
60/60 [==============================] - 0s 4ms/step - loss: 4.1768 - accuracy: 0.2088 - val_loss: 4.3179 - val_accuracy: 0.1494
Epoch 72/100
60/60 [==============================] - 0s 4ms/step - loss: 4.1770 - accuracy: 0.2100 - val_loss: 4.3181 - val_accuracy: 0.1426
Epoch 73/100
60/60 [==============================] - 0s 4ms/step - loss: 4.1649 - accuracy: 0.2119 - val_loss: 4.3146 - val_accuracy: 0.1445
Epoch 74/100
60/60 [==============================] - 0s 4ms/step - loss: 4.1799 - accuracy: 0.2016 - val_loss: 4.3142 - val_accuracy: 0.1426
Epoch 75/100
60/60 [==============================] - 0s 4ms/step - loss: 4.1701 - accuracy: 0.2111 - val_loss: 4.3111 - val_accuracy: 0.1426
Epoch 

Epoch 81/100
60/60 [==============================] - 0s 4ms/step - loss: 4.4446 - accuracy: 0.1334 - val_loss: 4.4884 - val_accuracy: 0.1016
Epoch 82/100
60/60 [==============================] - 0s 4ms/step - loss: 4.4358 - accuracy: 0.1363 - val_loss: 4.4871 - val_accuracy: 0.1006
Epoch 83/100
60/60 [==============================] - 0s 4ms/step - loss: 4.4370 - accuracy: 0.1408 - val_loss: 4.4854 - val_accuracy: 0.1016
Epoch 84/100
60/60 [==============================] - 0s 4ms/step - loss: 4.4357 - accuracy: 0.1394 - val_loss: 4.4842 - val_accuracy: 0.1045
Epoch 85/100
60/60 [==============================] - 0s 4ms/step - loss: 4.4290 - accuracy: 0.1431 - val_loss: 4.4834 - val_accuracy: 0.1016
Epoch 86/100
60/60 [==============================] - 0s 4ms/step - loss: 4.4281 - accuracy: 0.1457 - val_loss: 4.4815 - val_accuracy: 0.1045
Epoch 87/100
60/60 [==============================] - 0s 4ms/step - loss: 4.4260 - accuracy: 0.1444 - val_loss: 4.4808 - val_accuracy: 0.1055
Epoch 

Epoch 93/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5460 - accuracy: 0.0918 - val_loss: 4.5683 - val_accuracy: 0.0596
Epoch 94/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5476 - accuracy: 0.0905 - val_loss: 4.5677 - val_accuracy: 0.0596
Epoch 95/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5465 - accuracy: 0.0896 - val_loss: 4.5672 - val_accuracy: 0.0605
Epoch 96/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5490 - accuracy: 0.0875 - val_loss: 4.5666 - val_accuracy: 0.0615
Epoch 97/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5450 - accuracy: 0.0909 - val_loss: 4.5659 - val_accuracy: 0.0654
Epoch 98/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5440 - accuracy: 0.0882 - val_loss: 4.5655 - val_accuracy: 0.0664
Epoch 99/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5445 - accuracy: 0.0891 - val_loss: 4.5648 - val_accuracy: 0.0645
Epoch 

...

60/60 [==============================] - 0s 4ms/step - loss: 5.0556 - accuracy: 0.0164 - val_loss: 4.9610 - val_accuracy: 0.0117
Epoch 6/100
60/60 [==============================] - 0s 4ms/step - loss: 4.9379 - accuracy: 0.0179 - val_loss: 4.8707 - val_accuracy: 0.0117
Epoch 7/100
60/60 [==============================] - 0s 4ms/step - loss: 4.8555 - accuracy: 0.0176 - val_loss: 4.8057 - val_accuracy: 0.0156
Epoch 8/100
60/60 [==============================] - 0s 4ms/step - loss: 4.7964 - accuracy: 0.0222 - val_loss: 4.7589 - val_accuracy: 0.0156
Epoch 9/100
60/60 [==============================] - 0s 4ms/step - loss: 4.7521 - accuracy: 0.0208 - val_loss: 4.7242 - val_accuracy: 0.0186
Epoch 10/100
60/60 [==============================] - 0s 4ms/step - loss: 4.7197 - accuracy: 0.0224 - val_loss: 4.6985 - val_accuracy: 0.0186
Epoch 11/100
60/60 [==============================] - 0s 4ms/step - loss: 4.6950 - accuracy: 0.0264 - val_loss: 4.6793 - val_accuracy: 0.0195
Epoch 12/100
60/60 [===

Epoch 63/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5589 - accuracy: 0.0683 - val_loss: 4.5744 - val_accuracy: 0.0557
Epoch 64/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5592 - accuracy: 0.0684 - val_loss: 4.5738 - val_accuracy: 0.0566
Epoch 65/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5549 - accuracy: 0.0710 - val_loss: 4.5732 - val_accuracy: 0.0566
Epoch 66/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5547 - accuracy: 0.0767 - val_loss: 4.5726 - val_accuracy: 0.0547
Epoch 67/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5542 - accuracy: 0.0743 - val_loss: 4.5719 - val_accuracy: 0.0557
Epoch 68/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5515 - accuracy: 0.0755 - val_loss: 4.5713 - val_accuracy: 0.0557
Epoch 69/100
60/60 [==============================] - 0s 4ms/step - loss: 4.5534 - accuracy: 0.0727 - val_loss: 4.5706 - val_accuracy: 0.0576
Epoch 

Epoch 75/100
60/60 [==============================] - 0s 4ms/step - loss: 4.2646 - accuracy: 0.2316 - val_loss: 4.4137 - val_accuracy: 0.1475
Epoch 76/100
60/60 [==============================] - 0s 4ms/step - loss: 4.2493 - accuracy: 0.2364 - val_loss: 4.4103 - val_accuracy: 0.1514
Epoch 77/100
60/60 [==============================] - 0s 4ms/step - loss: 4.2440 - accuracy: 0.2408 - val_loss: 4.4091 - val_accuracy: 0.1426
Epoch 78/100
60/60 [==============================] - 0s 4ms/step - loss: 4.2490 - accuracy: 0.2383 - val_loss: 4.4097 - val_accuracy: 0.1484
Epoch 79/100
60/60 [==============================] - 0s 4ms/step - loss: 4.2413 - accuracy: 0.2364 - val_loss: 4.4044 - val_accuracy: 0.1504
Epoch 80/100
60/60 [==============================] - 0s 4ms/step - loss: 4.2341 - accuracy: 0.2388 - val_loss: 4.4048 - val_accuracy: 0.1494
Epoch 81/100
60/60 [==============================] - 0s 4ms/step - loss: 4.2409 - accuracy: 0.2412 - val_loss: 4.4038 - val_accuracy: 0.1475
Epoch 

Epoch 87/100
60/60 [==============================] - 0s 4ms/step - loss: 3.3463 - accuracy: 0.5168 - val_loss: 4.1585 - val_accuracy: 0.2061
Epoch 88/100
60/60 [==============================] - 0s 4ms/step - loss: 3.3264 - accuracy: 0.5315 - val_loss: 4.1683 - val_accuracy: 0.1953
Epoch 89/100
60/60 [==============================] - 0s 4ms/step - loss: 3.3392 - accuracy: 0.5213 - val_loss: 4.1727 - val_accuracy: 0.2002
Epoch 90/100
60/60 [==============================] - 0s 4ms/step - loss: 3.3220 - accuracy: 0.5235 - val_loss: 4.1625 - val_accuracy: 0.2002
Epoch 91/100
60/60 [==============================] - 0s 4ms/step - loss: 3.3249 - accuracy: 0.5217 - val_loss: 4.1599 - val_accuracy: 0.1982
Epoch 92/100
60/60 [==============================] - 0s 4ms/step - loss: 3.3030 - accuracy: 0.5283 - val_loss: 4.1726 - val_accuracy: 0.1953
Epoch 93/100
60/60 [==============================] - 0s 4ms/step - loss: 3.2786 - accuracy: 0.5393 - val_loss: 4.1651 - val_accuracy: 0.1992
Epoch 

Epoch 99/100
60/60 [==============================] - 0s 4ms/step - loss: 3.3863 - accuracy: 0.4702 - val_loss: 4.0861 - val_accuracy: 0.2070
Epoch 100/100
60/60 [==============================] - 0s 4ms/step - loss: 3.3906 - accuracy: 0.4722 - val_loss: 4.0799 - val_accuracy: 0.2100
No.19 hyperparameters search finished!!!, it takes 24.79 seconds
Current Best Train Accuracy:  0.5623697638511658
Current Best Valid Accuracy:  0.212890625
Current Best Hyperparameters:  {'learning_rate': 3.860778455077348e-05, 'l2_norm': 0.005014137944679996, 'M': 4, 'epsilon': 7.467785551056327e-06}

No.20 hyperparameters search started
Trying Hyperparameters: {'learning_rate': 2.6362063226356797e-06, 'l2_norm': 0.008884618333059232, 'M': 3, 'epsilon': 4.8642017717881164e-06}
Epoch 1/100
60/60 [==============================] - 1s 8ms/step - loss: 6.6973 - accuracy: 0.0088 - val_loss: 5.9674 - val_accuracy: 0.0098
Epoch 2/100
60/60 [==============================] - 0s 4ms/step - loss: 5.8048 - accuracy: